In [99]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')
# from sklearn.preprocessing import LabelEncoder

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# from sklearn import metrics
# from sklearn.metrics import roc_curve
# from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

#I'll set our seed here so our results can be consistent
np.random.seed(3141592)


In [100]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Customer ID is useless to us

Churn is our dependent variable so we will have to exclude it in our training set

Many of these columns are either binary choices, or categorical. Since the majority of them are binary i'll label encode the binary ones and use one hot encoding on the categories.

# Data Cleaning


In [101]:
df = df.drop(['customerID'], axis = 1)
df.drop(labels=df[df['tenure'] == 0].index, axis=0, inplace=True)
df["Partner"]= df["Partner"].map({"Yes": 1, "No": 0, "Male": 1, "Female": 0})
df["Dependents"]= df["Dependents"].map({"Yes": 1, "No": 0, "Male": 1, "Female": 0})
df["PhoneService"]= df["PhoneService"].map({"Yes": 1, "No": 0, "Male": 1, "Female": 0})
df["PaperlessBilling"]= df["PaperlessBilling"].map({"Yes": 1, "No": 0, "Male": 1, "Female": 0})
df["gender"]= df["gender"].map({"Yes": 1, "No": 0, "Male": 1, "Female": 0})

# Seperating the data sets

In [102]:
y = df['Churn']
X = df.drop(columns=['Churn'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) #Running the split function twice to seperate the data into 3 sets, Split is .6 Train, .2 Test, .2 Validation/holdout

Ill set up a pipeline to handle binary, categorical, and numerical data seperately

In [103]:
for col in df:
    print(f" '{col}': {df[col].unique()}")

 'gender': [0 1]
 'SeniorCitizen': [0 1]
 'Partner': [1 0]
 'Dependents': [0 1]
 'tenure': [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39]
 'PhoneService': [0 1]
 'MultipleLines': ['No phone service' 'No' 'Yes']
 'InternetService': ['DSL' 'Fiber optic' 'No']
 'OnlineSecurity': ['No' 'Yes' 'No internet service']
 'OnlineBackup': ['Yes' 'No' 'No internet service']
 'DeviceProtection': ['No' 'Yes' 'No internet service']
 'TechSupport': ['No' 'Yes' 'No internet service']
 'StreamingTV': ['No' 'Yes' 'No internet service']
 'StreamingMovies': ['No' 'Yes' 'No internet service']
 'Contract': ['Month-to-month' 'One year' 'Two year']
 'PaperlessBilling': [1 0]
 'PaymentMethod': ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
 'MonthlyCharges': [29.85 56.95 53.85 ... 63.1  44.2  78.7 ]

In [104]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
#Seperation of data types
num_attr = ['tenure', 'MonthlyCharges', 'TotalCharges']
bin_attr = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
cat_attr = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

print(cat_attr)
#Transformers

#imputer = SimpleImputer(strategy="median") #In the case that we want to do more scaling later

num_transformer_pipeline = Pipeline([
        #('imputer', SimpleImputer(strategy="median")), #In the case that we want to do more scaling later
        ('std_scaler', StandardScaler())
    ])

bin_transformer = 'passthrough'#Did this in the data cleaning step, so this will tell ColumnTransformer() to not change anything
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers = [
        ('num', num_transformer_pipeline, num_attr),
        ('bin', bin_transformer, bin_attr),
        ('cat', cat_transformer, cat_attr)
    ]
)

X_train_transformed = preprocessor.fit_transform(X_train)


['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [105]:
X_train_transformed
#Our training set is now transformed and ready for models

array([[-0.5799284 , -1.48699233, -0.84404594, ...,  0.        ,
         0.        ,  1.        ],
       [-1.10856811, -1.48699233, -0.95370345, ...,  0.        ,
         0.        ,  1.        ],
       [-1.27122649, -0.16287078, -0.9742012 , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.61595967,  1.68489582,  2.69353784, ...,  0.        ,
         1.        ,  0.        ],
       [-1.1898973 , -0.48806436, -0.94039635, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.57529507,  1.30466948,  2.22877589, ...,  1.        ,
         0.        ,  0.        ]])